In [171]:
import requests
import numpy as np
import pandas as pd
from pycoingecko import CoinGeckoAPI 
from bs4 import BeautifulSoup
import lxml
import html5lib
from selenium import webdriver
from datetime import datetime, timedelta

In [183]:
apikey = 'RF78TKBA1N4KIBXYBYJZ8JT6T7XWAUTNV7'
endpoint = "https://api.snowtrace.io/api"

cg = CoinGeckoAPI()
avax_price = cg.get_price(ids='avalanche-2', vs_currencies='usd')
avax_price = avax_price['avalanche-2']['usd']


daowallet = '0xEE576ebC6727D7Da876eDCd0429CdB48f69c3676'
daowallet2 = '0x0B4F85B173FA22f69308fFDD00bc556E987002df'
solanadao = 'dYG8h8sMrJHwPRdjAv85JGpj2Z44XFnFcHmADpJQhgp'

wei = 1000000000000000000

In [186]:
def get_multi_balance(multi_address):
    params = {
        "module": "account",
        "action": "balancemulti",
        "address": multi_address,
        "tag": "latest",
        "apikey": apikey
    }


    response = requests.get(endpoint, params=params)


    if response.status_code == 200:
        data = response.json()["result"]
        return data
    else:
        return "Request failed with status code:", response.status_code

def get_nfts(address, contract_address):
    params = {
        "module": "account",
        "action": "tokennfttx",
        "contractaddress": contract_address,
        "address": address,
        "page": "1",
        "offset": "100",
        "startblock": "0",
        "endblock": "99999999",
        "sort": "asc",
        "apikey": apikey
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()["result"]
        return data
    else:
        return "Request failed with status code:", response.status_code

def get_all_nfts(address):
    params = {
        "module": "account",
        "action": "tokennfttx",
        "address": address,
        "page": "1",
        "offset": "100",
        "startblock": "0",
        "endblock": "99999999",
        "sort": "asc",
        "apikey": apikey
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()["result"]
        return data
    else:
        return "Request failed with status code:", response.status_code
        

In [192]:
multi_address = daowallet,daowallet2
balance_wallets = get_multi_balance(multi_address)
balance_wallets = pd.DataFrame(balance_wallets)

balance_wallets


,account,balance
0,0xEE576ebC6727D7Da876eDCd0429CdB48f69c3676,627014954314272191286
1,0x0B4F85B173FA22f69308fFDD00bc556E987002df,595171108398898991


In [193]:
balance_wallets['balance'] = balance_wallets['balance'].astype(np.float64)
balance_wallets['balance'] = balance_wallets['balance']/wei
balance_wallets

,account,balance
0,0xEE576ebC6727D7Da876eDCd0429CdB48f69c3676,627.014954
1,0x0B4F85B173FA22f69308fFDD00bc556E987002df,0.595171


In [194]:
new_row = pd.DataFrame({'account': ['stakedjoe'], 'balance': [1700]})
balance_wallets = pd.concat([balance_wallets, new_row], ignore_index=True)
balance_wallets.rename(columns={'balance':'balance_avax'}, inplace=True)

In [195]:
balance_wallets['balance_usd'] = balance_wallets['balance_avax'] * avax_price
balance_wallets['balance_avax'] = balance_wallets['balance_avax'].apply(lambda x: round(x, 2))
balance_wallets['balance_usd'] = balance_wallets['balance_usd'].apply(lambda x: round(x, 2))

balance_wallets.to_csv("balance.csv", index=True)
balance_wallets


,account,balance_avax,balance_usd
0,0xEE576ebC6727D7Da876eDCd0429CdB48f69c3676,627.01,10809.74
1,0x0B4F85B173FA22f69308fFDD00bc556E987002df,0.60,10.26
2,stakedjoe,1700.00,29308.00


In [142]:
daowalletdf = get_all_nfts(daowallet)
daowalletdf = pd.DataFrame(daowalletdf)

daowallet2df = get_all_nfts(daowallet2)
daowallet2df = pd.DataFrame(daowallet2df)

nfts = pd.concat([daowalletdf, daowallet2df], ignore_index=True)
nfts

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,tokenID,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
0,22162499,1668013188,0x4dae6abc0c7cd8bc13e0ba4f977d735d56e6000bd64d...,39,0xa7b431dfced351ea4bbe73448c6af7aa4960e3d8afcc...,0xa4f9a95bef3d922d1f0f8a4d85396775c6c886eb,0x1869f12d1eb4a1240ddcb259b0b155d935ff2122,0xee576ebc6727d7da876edcd0429cdb48f69c3676,621,Ouroboros Club,OUROBOROS-CLUB,0,6,593882,26500000000,592434,1526073,deprecated,4047174
1,22162499,1668013188,0x4dae6abc0c7cd8bc13e0ba4f977d735d56e6000bd64d...,39,0xa7b431dfced351ea4bbe73448c6af7aa4960e3d8afcc...,0xa4f9a95bef3d922d1f0f8a4d85396775c6c886eb,0x1869f12d1eb4a1240ddcb259b0b155d935ff2122,0xee576ebc6727d7da876edcd0429cdb48f69c3676,620,Ouroboros Club,OUROBOROS-CLUB,0,6,593882,26500000000,592434,1526073,deprecated,4047174
2,22162499,1668013188,0x4dae6abc0c7cd8bc13e0ba4f977d735d56e6000bd64d...,39,0xa7b431dfced351ea4bbe73448c6af7aa4960e3d8afcc...,0xa4f9a95bef3d922d1f0f8a4d85396775c6c886eb,0x880fe52c6bc4ffffb92d6c03858c97807a900691,0xee576ebc6727d7da876edcd0429cdb48f69c3676,1400,Party Animals,APA,0,6,593882,26500000000,592434,1526073,deprecated,4047174
3,22162499,1668013188,0x4dae6abc0c7cd8bc13e0ba4f977d735d56e6000bd64d...,39,0xa7b431dfced351ea4bbe73448c6af7aa4960e3d8afcc...,0xa4f9a95bef3d922d1f0f8a4d85396775c6c886eb,0x3025c5c2aa6eb7364555aac0074292195701bbd6,0xee576ebc6727d7da876edcd0429cdb48f69c3676,3318,MadSkullz,MADSKULLZ,0,6,593882,26500000000,592434,1526073,deprecated,4047174
4,22162499,1668013188,0x4dae6abc0c7cd8bc13e0ba4f977d735d56e6000bd64d...,39,0xa7b431dfced351ea4bbe73448c6af7aa4960e3d8afcc...,0xa4f9a95bef3d922d1f0f8a4d85396775c6c886eb,0xa09477b44cff7b525436364a0da4e870bc9e052c,0xee576ebc6727d7da876edcd0429cdb48f69c3676,822,Ripperz,RIPPERZ,0,6,593882,26500000000,592434,1526073,deprecated,4047174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,22928275,1669594957,0xce8bc915ecf3790865cfba847297027c4486bea8caa6...,7,0x521dca497b228ad558f22a6e3ea7a5611fc2e24dbdf9...,0x0000000000000000000000000000000000000000,0xc97b528ae24937175fa66f7d3634ac1049c5c79c,0x0b4f85b173fa22f69308ffdd00bc556e987002df,94,Black Swan Club,BSC,0,1,1019434,27352250000,1003949,1622366,deprecated,3281398
146,22928275,1669594957,0xce8bc915ecf3790865cfba847297027c4486bea8caa6...,7,0x521dca497b228ad558f22a6e3ea7a5611fc2e24dbdf9...,0x0000000000000000000000000000000000000000,0xc97b528ae24937175fa66f7d3634ac1049c5c79c,0x0b4f85b173fa22f69308ffdd00bc556e987002df,95,Black Swan Club,BSC,0,1,1019434,27352250000,1003949,1622366,deprecated,3281398
147,22928275,1669594957,0xce8bc915ecf3790865cfba847297027c4486bea8caa6...,7,0x521dca497b228ad558f22a6e3ea7a5611fc2e24dbdf9...,0x0000000000000000000000000000000000000000,0xc97b528ae24937175fa66f7d3634ac1049c5c79c,0x0b4f85b173fa22f69308ffdd00bc556e987002df,96,Black Swan Club,BSC,0,1,1019434,27352250000,1003949,1622366,deprecated,3281398
148,22928275,1669594957,0xce8bc915ecf3790865cfba847297027c4486bea8caa6...,7,0x521dca497b228ad558f22a6e3ea7a5611fc2e24dbdf9...,0x0000000000000000000000000000000000000000,0xc97b528ae24937175fa66f7d3634ac1049c5c79c,0x0b4f85b173fa22f69308ffdd00bc556e987002df,97,Black Swan Club,BSC,0,1,1019434,27352250000,1003949,1622366,deprecated,3281398


In [143]:
date_nfts = nfts
date_nfts.drop(columns=['hash', 'from', 'blockNumber', 'nonce', 'blockHash', 'contractAddress', 'to', 'tokenSymbol', 'tokenDecimal', 'transactionIndex', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'input', 'confirmations'], inplace=True)
date_nfts['timeStamp'] = pd.to_datetime(date_nfts['timeStamp'], unit='s')
date_nfts.rename(columns={'timeStamp': 'date', 'tokenName' : 'Name'}, inplace=True)
date_nfts.to_csv("date_nfts.csv", index=True)
date_nfts

,date,tokenID,Name
0,2022-11-09 16:59:48,621,Ouroboros Club
1,2022-11-09 16:59:48,620,Ouroboros Club
2,2022-11-09 16:59:48,1400,Party Animals
3,2022-11-09 16:59:48,3318,MadSkullz
4,2022-11-09 16:59:48,822,Ripperz
...,...,...,...
145,2022-11-28 00:22:37,94,Black Swan Club
146,2022-11-28 00:22:37,95,Black Swan Club
147,2022-11-28 00:22:37,96,Black Swan Club
148,2022-11-28 00:22:37,97,Black Swan Club


In [144]:
data = [
    {'Name': 'Black Swan Club', 'Price': 0.35, 'Chain': 'Avalanche'},
    {'Name': 'CHAD DOGE', 'Price': 2.58, 'Chain': 'Avalanche'},
    {'Name': 'Ouroboros Club', 'Price': 2.5, 'Chain': 'Avalanche'},
    {'Name': 'chikn', 'Price': 40, 'Chain': 'Avalanche'},
    {'Name': 'FarmLand', 'Price': 8, 'Chain': 'Avalanche'},
    {'Name': 'Chad Doge Supers', 'Price': 3.19, 'Chain': 'Avalanche'},
    {'Name': 'Party Animals', 'Price': 5.35, 'Chain': 'Avalanche'},
    {'Name': 'MadSkullz', 'Price': 3.35, 'Chain': 'Avalanche'},
    {'Name': 'Ripperz', 'Price': 2.44, 'Chain': 'Avalanche'},
    {'Name': 'Innermost by Brian Morris', 'Price': 3.45, 'Chain': 'Avalanche'},
    {'Name': 'Rich Peon Poor Peon', 'Price': 5.1, 'Chain': 'Avalanche'},
    {'Name': 'Wolfi Land', 'Price': 2, 'Chain': 'Avalanche'},
    {'Name': 'Gen 0 Shidizens', 'Price': 2, 'Chain': 'Avalanche'},
    {'Name': 'Smol Creeps', 'Price': 7.3, 'Chain': 'Avalanche'},
    {'Name': 'Beeg Pumpkins', 'Price': 37, 'Chain': 'Avalanche'},
    {'Name': 'Nox Beyond', 'Price': 1.8, 'Chain': 'Avalanche'},
]

price_nfts = pd.DataFrame(data, columns=['Name', 'Price', 'Chain'])
price_nfts.head()

,Name,Price,Chain
0,Black Swan Club,0.35,Avalanche
1,CHAD DOGE,2.58,Avalanche
2,Ouroboros Club,2.50,Avalanche
3,chikn,40.00,Avalanche
4,FarmLand,8.00,Avalanche


In [9]:
count_nfts = nfts['tokenName'].value_counts()
count_nfts = count_nfts.reset_index()
count_nfts = count_nfts.rename(columns={'index': 'Name', 'tokenName': 'Count'})


In [ ]:
nfts = pd.merge(price_nfts, count_nfts, on='Name')
nfts['Value'] = nfts['Price'] * nfts['Count']
nfts_sol = pd.DataFrame({
    'Name': ['y00ts'],
    'Price': [155],
    'Chain': ['Solana'],
    'Count': [1],
    'Value': [155]
})
nfts = pd.concat([nfts, nfts_sol], ignore_index=True)

nfts.to_csv("nfts.csv", index=True)

nfts

,Name,Price,Chain,Count,Value
0,Black Swan Club,0.35,Avalanche,124,43.40
1,CHAD DOGE,2.58,Avalanche,5,12.90
2,Ouroboros Club,2.50,Avalanche,2,5.00
3,chikn,40.00,Avalanche,2,80.00
4,FarmLand,8.00,Avalanche,2,16.00
5,Chad Doge Supers,3.19,Avalanche,2,6.38
6,Party Animals,5.35,Avalanche,1,5.35
7,MadSkullz,3.35,Avalanche,1,3.35
8,Ripperz,2.44,Avalanche,1,2.44
9,Innermost by Brian Morris,3.45,Avalanche,1,3.45


In [ ]:
nfts_avax = nfts[nfts['Chain'] == 'Avalanche']
nfts_avax.to_csv("nfts_avax.csv", index=True)
nfts_avax

,Name,Price,Chain,Count,Value
0,Black Swan Club,0.35,Avalanche,124,43.40
1,CHAD DOGE,2.58,Avalanche,5,12.90
2,Ouroboros Club,2.50,Avalanche,2,5.00
3,chikn,40.00,Avalanche,2,80.00
4,FarmLand,8.00,Avalanche,2,16.00
5,Chad Doge Supers,3.19,Avalanche,2,6.38
6,Party Animals,5.35,Avalanche,1,5.35
7,MadSkullz,3.35,Avalanche,1,3.35
8,Ripperz,2.44,Avalanche,1,2.44
9,Innermost by Brian Morris,3.45,Avalanche,1,3.45


In [152]:
avax_price = cg.get_coin_history_by_id(id='avalanche-2', date="30-12-2019")
avax_price

{'id': 'avalanche-2',
 'symbol': 'avax',
 'name': 'Avalanche',
 'localization': {'en': 'Avalanche',
  'de': 'Avalanche',
  'es': 'Avalanche',
  'fr': 'Avalanche',
  'it': 'Avalanche',
  'pl': 'Avalanche',
  'ro': 'Avalanche',
  'hu': 'Avalanche',
  'nl': 'Avalanche',
  'pt': 'Avalanche',
  'sv': 'Avalanche',
  'vi': 'Avalanche',
  'tr': 'Avalanche',
  'ru': 'Avalanche',
  'ja': 'アバランチ',
  'zh': 'Avalanche',
  'zh-tw': 'Avalanche',
  'ko': 'Avalanche',
  'ar': 'Avalanche',
  'th': 'Avalanche',
  'id': 'Avalanche',
  'cs': 'Avalanche',
  'da': 'Avalanche',
  'el': 'Avalanche',
  'hi': 'Avalanche',
  'no': 'Avalanche',
  'sk': 'Avalanche',
  'uk': 'Avalanche',
  'he': 'Avalanche',
  'fi': 'Avalanche',
  'bg': 'Avalanche',
  'hr': 'Avalanche',
  'lt': 'Avalanche',
  'sl': 'Avalanche'},
 'image': {'thumb': 'https://assets.coingecko.com/coins/images/12559/thumb/Avalanche_Circle_RedWhite_Trans.png?1670992574',
  'small': 'https://assets.coingecko.com/coins/images/12559/small/Avalanche_Circle_